# **8장. 성능 최적화**

# 8.1 성능 최적화

8.1.1 데이터를 사용한 최적화

    - 최대한 많은 데이터 수집하기
    - 데이터 생성하기 (이미지 조작)
    - 데이터 범위 조정하기 (scale)
    - 정규화, 규제화, 표준화 등

8.1.2 알고리즘을 이용한 성능 최적화

    - 알고리즘을 바꿔가면서 최적의 성능 보이는 알고리즘 선택

8.1.3 알고리즘 튜닝을 이용한 성능 최적화 (하이퍼파라미터)

    - 진단
    - 가중치
    - 학습률
    - 활성화 함수
    - 배치와 에포크
    - 옵티마이저 및 손실 함수
    - 네트워크 구성 (ex: 하나의 은닉층에 여러 뉴런(네트워크가 넓음), 여러 계층에 적은 뉴런(네트워크 깊음))

8.1.4 앙상블을 이용한 성능 최적화

    - 앙상블 : 모델을 두 개 이상 섞어서 사용하는 것

# 8.2 하드웨어를 이용한 성능 최적화 (CPU, GPU)

8.2.1 CPU와 GPU의 차이

    - CPU : 한 번에 하나의 명령어만 순차적으로 처리
    - GPU : 서로 다른 명령어를 동시에 병렬적으로 처리

# 8.3 하이퍼파라미터를 이용한 성능 최적화

    - 배치 정규화
    - 드롭 아웃
    - 조기 종료

In [1]:
#배치 정규화와 드롭아웃 예제

#라이브러리 호출
import torch
import matplotlib.pyplot as plt
import numpy as np

import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.optim as optim

In [2]:
trainset = torchvision.datasets.FashionMNIST(root='/content/drive/MyDrive/pytorch_ex/chap08/data',
                                             train=True, download=True, transform=transforms.ToTensor())

100%|██████████| 26.4M/26.4M [00:03<00:00, 8.51MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 210kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.90MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 12.7MB/s]


In [3]:
batch_size = 4
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

In [ ]:
#데이터셋 분리
dataiter = iter(trainloader)
images, labels = dataiter.next()

print(images.shape)